In [1]:
import numpy as np
import math
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
from odlib import *
import time

In [20]:
class OD:
    '''Class that performs all orbital determination calculations'''
    
    def __init__(self, inputFile:str):
        """ Initializes OD class
            Args:
                inputFile (str): input file name
            Returns:
                None
        """
        # constants
        self.k = 0.0172020989484 #Gaussian gravitational constant  
        self.cAU = 173.144643267 #speed of light in au/(mean solar)day  
        self.mu = 1
        self.eps = np.radians(23.4374) #Earth's obliquity
        self.toGaussian=365.2568983
        self.mu = 1
        
        self.data=Data() # Data object
        self.inputFile=inputFile
        
    def genElements(self, pos:list, vel:list, time:float, update:bool=True):
        """ Calculates and returns the orbital elements given position, velocity, time
            Args:
                pos (list): the position vector
                vel (list): the velocity vector
                time (float): the time in Julian days
                update (bool): if True keeps the newly calculated orbital elements
            Returns:
                floats: the orbital elements; a,e,i,o,v,w,T,M
        """
        if update:
            self.pos,self.vel,self.time=pos,vel,time
            self.od=ODElements(self.pos,self.vel,self.time)
            self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
            return self.getElements()
        else:
            od=ODElements(pos,vel,time)
            return od.getElements()
    
    def getElements(self):
        """ Returns the orbital elements (already calculated)
            Args:
                rad (bool): True if return in radians
            Returns:
                floats: a,e,i,o,v,w,T,M
        """
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M
    
    def printODElementsErr(self, file:str, date:str):
        """ Prints the OD elements and compares them to results (expected, calculated, % error)
            Args:
                file (str): actual values file name
                date (str): date that the OD elements were calculated for
            Returns:
                None
        """
        self.od.printError(self.data.getODActualData(file, date))
    
    def getT(self)->float:
        """ Returns the time of perihelion
            Args:
                None
            Returns:
                float: time of perihelion
        """
        return self.T # time of perihelion
    
    def SEL(self, taus:list, sunR2:list, rhohat2:list, coefD:list):
        """ Scalar Equation of Lagrange to calculate the roots (r) and rhos corresponding to each r
            Args:
                taus (list): a list of floats of taus [T1,T3,T]
                sunR2 (list): a list representing the sun vector R2
                rhohat2 (list): a list representing the rhohat2 vector
                coefD (list): a list of D coefficients [D0,D21,D22,D23]
            Returns:
                lists: roots (r's), rhos
        """
        T1,T3,T=taus[0],taus[1],taus[2]
        D0,D21,D22,D23=coefD[0],coefD[1],coefD[2],coefD[3]
        A1=T3/T
        B1=A1/6*(T**2-T3**2)
        A3=-T1/T
        B3=A3/6*(T**2-T1**2)
        A=(A1*D21-D22+A3*D23)/(-D0)
        B=(B1*D21+B3*D23)/(-D0)
        
        E=-2*(dot(rhohat2, sunR2))
        F=dot(sunR2,sunR2)
        
        a=-(A**2+A*E+F)
        b=-self.mu*(2*A*B+B*E)
        c=-self.mu**2*B**2
        
        coef=[c,0,0,b,0,0,a,0,1]
        res=poly.polyroots(coef)

        temproots=[]
        for val in res: 
            if np.isreal(val) and np.real(val)>0: temproots.append(np.real(val))

        temprhos=[A+B/temproots[i]**3 for i in range(len(temproots))]
        
        # ignore pairs where rho magnitude is negative
        roots=[]
        rhos=[]
        #print(temproots, temprhos)
        for i in range(len(temproots)):
            if temprhos[i]>0.0:
                roots.append(temproots[i])
                rhos.append(temprhos[i])
        
        return roots,rhos
    
    def ephemeris(self, time:float, date:str, sunFile:str):
        """ Calculates RA and Dec given time and date, using previously calculated orbital elements
            Args:
                time (float): time to determine ephemeris for in Julian Days
                date (str): date for which to determine ephemeris
                sunFile (str): file name for sun positions
            Returns:
                floats: ra, dec
        """
        n=self.k*math.sqrt(self.mu/(self.a**3))
        M=n*(time-self.T)
        E=newton(lambda E:M - E + self.e*np.sin(E), lambda E: -1+self.e*np.cos(E), M, 1e-14)
        
        pos=np.array([self.a*math.cos(E)-self.a*self.e, self.a*math.sqrt(1-self.e**2)*math.sin(E), 0])
        
        # the four rotations
        pos=rotZX(pos,np.deg2rad(self.w),np.deg2rad(self.i))
        pos=rotZX(pos,np.deg2rad(self.o),self.eps)
        
        R=self.data.getSunPos(date, sunFile)
        if np.array_equal(R, np.array([0,0,0])): raise Exception("Sun Position Not Found in SunPos.txt")
        rho=pos+R
        rhohat=rho/getMag(rho)
        
        dec=math.asin(rhohat[2])
        cra=rhohat[0]/math.cos(dec)
        sra=rhohat[1]/math.cos(dec)

        ra=getAngle(sra,cra)
        
        dec=np.rad2deg(dec)
        ra=np.rad2deg(ra)
        
        dec=DECdecimalToDMS(dec)
        ra=RAdecimalToHMS(ra)
        
        return ra,dec
        
        
    def fg(self, tau:float, r2mag:float, r2dot:list, order:int, r2:list=[]):
        """ Gets the f and g values given one time
            Args:
                tau (float): the time in Julian Days
                r2mag (float): the magnitude of r2
                r2dot (list): the velocity vector 2
                order (int): order of f and g taylor series approximations
                r2 (list): optional parameter, the position vector 2
            Returns:
                floats: f, g
        """
        if len(r2)==0: u=self.mu/r2mag**3
        else: u=self.mu/getMag(r2)**3
        
        f=1-1/2*u*tau**2
        g=tau
        
        if order>=3:
            z=dot(r2,r2dot)/(dot(r2,r2))
            q=dot(r2dot,r2dot)/(dot(r2,r2))-u
            f+=1/2*u*z*tau**3
            g+=-1/6*u*tau**3
        
        if order>=4:
            f+=1/24*(3*u*q-15*u*z**2+u**2)*tau**4
            g+=1/4*u*z*tau**4
        
        return f, g
        
    def getFGVals(self, tau1:float, tau3:float, r2mag:float, r2dot:list, order1:int, order2:int, r2:list=[]):
        """ Gets all f and g values
            Args:
                tau1 (float): the time in Julian Days for observation 1 from obs 2(T1-T2)
                tau3 (float): the time in Julian days for observation 3 from obs 2(T3-T2)
                r2mag (float): the magnitude of r2
                r2dot (list): the velocity vector 2
                order1 (int): Order of Taylor Series expansion for f and g values for observation 1
                order2 (int): Order of Taylor Series expansion for f and g values for observation 3
                r2 (list): optional parameter, the position vector 2
            Returns:
                lists: [f1,f3], [g1,g3]
        """
        f1,g1=self.fg(tau1,r2mag,r2dot,order1,r2)
        f3,g3=self.fg(tau3,r2mag,r2dot,order2,r2)
        return [f1,f3], [g1,g3]
    
    def getDCoef(self, ra:list, dec:list, R1:list, R2:list, R3:list):
        """ Gets the D coefficients given the ra and dec for three observations (in radians)
            Args:
                ra (list): the right ascensions for three observations (radians)
                dec (list): the declinations for three observations (radians)
                R1 (list): the sun vector for observation 1
                R2 (list): the sun vector for observation 2
                R3 (list): the sun vector for observation 3
            Returns:
                list: [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33], [rhohat1, rhohat2, rhohat3]
        """
        ra1,ra2,ra3=ra[0],ra[1],ra[2]
        dec1,dec2,dec3=dec[0],dec[1],dec[2]
        
        rhohat1=np.array([np.cos(ra1)*np.cos(dec1), np.sin(ra1)*np.cos(dec1), np.sin(dec1)])
        rhohat2=np.array([np.cos(ra2)*np.cos(dec2), np.sin(ra2)*np.cos(dec2), np.sin(dec2)])
        rhohat3=np.array([np.cos(ra3)*np.cos(dec3), np.sin(ra3)*np.cos(dec3), np.sin(dec3)])
        
        D0=dot(rhohat1, cross(rhohat2,rhohat3))
        D11=dot(cross(R1, rhohat2),rhohat3)
        D12=dot(cross(R2, rhohat2),rhohat3)
        D13=dot(cross(R3, rhohat2),rhohat3)
        
        D21=dot(cross(rhohat1,R1), rhohat3)
        D22=dot(cross(rhohat1,R2), rhohat3)
        D23=dot(cross(rhohat1,R3), rhohat3)
        
        D31=dot(rhohat1, cross(rhohat2,R1))
        D32=dot(rhohat1, cross(rhohat2,R2))
        D33=dot(rhohat1, cross(rhohat2,R3))
     
        return [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33], np.array([rhohat1, rhohat2, rhohat3])
    
    def MoGGenData(self,selTime:list=[],selDate:list=[]):
        """ Generates the data for Method of Gauss calculations
            Args:
                selTime (list): list of chosen times (in Julian days) for observations 1,2,3
                selDate (list): list of chosen datesfor observations 1,2,3
            Returns:
                lists: ra, dec, R1, R2, R3, taus, ts (original times in Julian days)
        """
        self.data.getInput(self.inputFile) # formats and generates info
        # generate data
        if not(selTime==[]): # using Julian day times
            ra,dec=[],[]
            for time in selTime:
                r,d=self.data.getRADECInput(time=time)
                ra.append(np.deg2rad(r))
                dec.append(np.deg2rad(d))
            R1=self.data.getSunInput(time=selTime[0])
            R2=self.data.getSunInput(time=selTime[1])
            R3=self.data.getSunInput(time=selTime[2])
            # calculate the taus
            t1,t2,t3=selTime[0],selTime[1],selTime[2]
            ts=[t1,t2,t3]
            T1=t1-t2
            T3=t3-t2
            T=t3-t1
            taus = [T1,T3,T] # in Gaussian days
                
        elif not(selDate==[]):
            ra,dec=[],[]
            for date in selDate:
                r,d=self.data.getRADECInput(date=date)
                ra.append(np.deg2rad(r))
                dec.append(np.deg2rad(d))
            R1=self.data.getSunInput(date=selDate[0])
            R2=self.data.getSunInput(date=selDate[1])
            R3=self.data.getSunInput(date=selDate[2])
            # calculate the taus given the dates
            t1,t2,t3=self.data.getJDTime(selDate[0]),self.data.getJDTime(selDate[1]),self.data.getJDTime(selDate[2])
            ts=[t1,t2,t3]
            T1=t1-t2
            T3=t3-t2
            T=t3-t1
            taus = [T1*self.k,T3*self.k,T*self.k]
            
        else: raise Exception("No data given")
        
        return ra,dec,np.array(R1),np.array(R2),np.array(R3),taus,np.array(ts)
    
    def MoGCalcRhos(self, rhohats:list, coefD:list, fvals:list, gvals:list):
        """ Generates the rhos (and their magnitudes) for Method of Gauss calculations
            Args:
                rhohats (list): the direction vectors for rhos
                coefD (list): the D coefficients
                fvals (list): the f values
                gvals (list): the g values
            Returns:
                lists: rhos, rhomags
        """
        f1,f3=fvals
        g1,g3=gvals
        D0,D11,D12,D13,D21,D22,D23,D31,D32,D33=coefD
        C1=g3/(f1*g3-g1*f3)
        C2=-1
        C3=-g1/(f1*g3-g1*f3)
        rho1=(C1*D11+C2*D12+C3*D13)/(C1*D0)
        rho2=(C1*D21+C2*D22+C3*D23)/(C2*D0)
        rho3=(C1*D31+C2*D32+C3*D33)/(C3*D0)
        rhomags=np.array([rho1,rho2,rho3])
        rhos=np.array([rhohats[0]*rho1, rhohats[1]*rho2, rhohats[2]*rho3])
        return rhos, rhomags
    
    def MoGCalcPos(self, rhos:list, Rs:list)->list:    
        """ Calculates the r vectors (position of asteroid from sun) for Method of Gauss calculations
            Args:
                rhos (list): the position from Earth vectors
                Rs (list): the position from Sun vectors
            Returns:
                lists: rs
        """
        return rhos-Rs
        
    def MoGGetErr(self, prev:list, cur:list, tolerance:float)->bool:
        """ Calculates the r vectors (position of asteroid from sun) for Method of Gauss calculations
            Args:
                prev (list): previous r vector
                cur (list): the newly calculated r vector
                tolerance (float): the tolerance
            Returns:
                bool: True if error is good, False if no within tolerance
        """
        for i in range(len(prev)):
            if abs(prev[i]-cur[i])>tolerance: return False
        return True
    
    def MoGGetAdjustedTaus(self, origt:list, rhomags:float)->list:
        """ Returns adjusted taus for Method of Gauss calculations
            Args:
                origt (list): original observation times in Julian days
                rhomags (list): the magnitude of the rho vectors
            Returns:
                list: the adjusted taus
        """
        ts=np.copy(origt)-rhomags/self.cAU
        t1,t2,t3=ts
        return [(t1-t2)*self.k,(t3-t2)*self.k,(t3-t1)*self.k]
    
    def MoG(self,selTime:list=[],selDate:list=[]):
        """ Performs Method of Gauss calculations to determine orbital elements
            Args:
                selTime (list): list of chosen times (in Julian days) for observations 1,2,3
                selDate (list): list of chosen datesfor observations 1,2,3
            Returns:
                None
        """
        # generate data
        ra,dec,R1,R2,R3,taus,ts=self.MoGGenData(selTime,selDate)
        
        # calculate the initial estimate for r2 DONE
        coefD,rhohats=self.getDCoef(ra, dec, R1, R2, R3)

        SELcoefD=[coefD[0],coefD[4],coefD[5],coefD[6]] # [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33] -> [D0,D21,D22,D23]
        r2MagGuesses,rhoMagGuesses=self.SEL(taus, R2, rhohats[1], SELcoefD)
        
        results=[]
        
        # calculate the f and g values to second order
        for r2mag in r2MagGuesses:
            # initial guesses
            fvals,gvals=self.getFGVals(taus[0], taus[1], r2mag, [], 2, 2) # using r2mag
            rhos,rhomags=self.MoGCalcRhos(rhohats, coefD, fvals, gvals)
            rs=self.MoGCalcPos(rhos, np.array([R1,R2,R3]))
   
            # calculate the central velocity vector
            f1,f3=fvals
            g1,g3=gvals
            r2dot = (-f3/(-f3*g1+g3*f1))*rs[0] + (f1/(f1*g3-f3*g1))*rs[2]
            counter=0
            timedOut=True
            timeout=time.time() + 60*3 # timeout after 3 minutes
            
            while time.time()<timeout: # timeout
                prev=rs[1]
                
                # time adjustment
                newtaus=self.MoGGetAdjustedTaus(ts, rhomags)
                
                fvals,gvals=self.getFGVals(newtaus[0], newtaus[1], r2mag, r2dot, 4, 4, rs[1]) # using r2mag
                rhos,rhomags=self.MoGCalcRhos(rhohats, coefD, fvals, gvals)
                rs=self.MoGCalcPos(rhos, np.array([R1,R2,R3]))
                
                f1,f3=fvals
                g1,g3=gvals
                r2dot = (-f3/(-f3*g1+g3*f1))*rs[0] + (f1/(f1*g3-f3*g1))*rs[2]
                
                counter+=1
                
                if (self.MoGGetErr(prev, rs[1], 1e-23)): 
                    timedOut=False
                    break
            
            if timedOut: print("Timed out")
            else: 
                # rotate around x axis to get to ecliptic
                r2=rotX(np.array(rs[1]),-self.eps)
                r2dot=rotX(r2dot,-self.eps)
                rho2=rotX(rhos[1],-self.eps)
                results.append([r2, r2dot, rho2]) # position and velocity
        
        for result in results:
            pos,vel,rho=result
            self.genElements(pos,np.array(vel)*(2*math.pi)/365.2568983,ts[1])
            
        self.vel = vel
        self.pos = pos
        self.rho = rho
        
    def getError(self, results:list):
        """ Prints error and results from orbital elements determination after Method of Gauss calculations
            Args:
                results (list): [e,0,i,o,w,T,0,0,0,a]
            Returns:
                None
        """
        self.od.printError(results)
        
    def exportResults(self, time:float, date:str, fileName:str):
        """ Exports results to a file
            Args:
                time (float): time in Julian days for the Mean Anomaly
                date (str): date for Mean Anomaly
                fileName (str): the exported file name
            Returns:
                None
        """
        # calculate mean anomaly for July 24, 2022
        n=self.k*math.sqrt(self.mu/(self.a**3))
        M=n*(time-self.T)
        self.data.printResults(fileName, self.pos, self.vel, self.rho, self.a, self.e, self.i, self.o,self.T,self.w,date,M)
        
        

    

In [21]:
def gen1999GJ2():
    file = "/home/soonali/Desktop/SSP2022/SoongInput.txt"
    data=Data()
    od=OD(file)
    od.MoG(selDate=['2022-06-28 04:33:44.089','2022-07-12 04:16:40.826','2022-07-14 04:41:39.025'])
    a,e,i,o,w,m,t= 1.535501123505049E+00, 1.980131722243141E-01, 1.127908127242095E+01, 1.961976310756240E+02, 1.425325538303718E+02, 3.163937598615976E+02, 2459856.859881772194
    od.getError([e,0,i,o,w,t,0,0,0,a])
    
    od.exportResults(2459784.7916667,"July 24, 2022 7:00 UT", "results.txt")
    
    return od

od = gen1999GJ2()

Semi-major axis: 
	actual: 1.535501123505049 
	calculated: 1.5347326380022122 
	error: 0.050047863272331704
Eccentricity: 
	actual: 0.1980131722243141 
	calculated: 0.1970528139135287 
	error: 0.48499718478198134
Inclination: 
	actual: 11.27908127242095 
	calculated: 11.297805611270112 
	error: 0.16600943283338615
Longitude of Ascending Node: 
	actual: 196.197631075624 
	calculated: 196.30891625203841 
	error: 0.05672095825230317
Argument of perihelion: 
	actual: 142.5325538303718 
	calculated: 142.4976525114411 
	error: 0.024486559731640926
Time of Perihelion Passage T: 
	actual: 2459856.859881772 
	calculated: 2459162.6487638657 
	error: 0.028221606274271654


In [15]:
def testGenOD():
    # test case from final OD instructions
    file = "/home/soonali/Desktop/SSP2022/TestODElements.txt"
    data=Data()
    od=OD(file)
    od.MoG(selDate=['2021 06 25 00:00:00.000','2021 07 05 00:00:00.000','2021 07 15 00:00:00.000'])
    print(od.getElements()) # a,e,i,o,v,w,T,M
    
testGenOD()

(2.720849390742201, 0.53665552802283, 3.8972714861618827, 238.47520894802534, 292.9064844772765, 108.23228186027606, 2457853.3904677215, 339.75697548706864)


In [16]:
def testEphemeris(od):
    newdate = '2022-Jul-18 04:00:00'
    newjdtime = 2459778.6666667
    sunFile="/home/soonali/Desktop/SSP2022/1999GJ2SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 23 15.97',"\n\terror:",error(HMStoDeg(16,23,15.97),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+10 37 45.1',"\n\terror:",error(DMStoDeg(10,37,45.1),DMStoDeg(dec[0],dec[1],dec[2])))


testEphemeris(od)

ra: 
	calculated: (16.0, 23.0, 14.304874179933904) 
	actual: 16 23 15.97 
	error: 0.0028224399396533286
dec: 
	calculated: (10, 37, 35.77783100793983) 
	actual: +10 37 45.1 
	error: 0.02436206619625889


In [17]:
def testJPL():
    file = "/home/soonali/Desktop/SSP2022/JPLInput.txt"
    data=Data()
    od=OD(file)
    od.MoG(selDate=['2022-06-28 04:33:44.089','2022-07-12 04:16:40.826','2022-07-14 04:41:39.025'])
    a,e,i,o,w,m,t= 1.535501123505049E+00, 1.980131722243141E-01, 1.127908127242095E+01, 1.961976310756240E+02, 1.425325538303718E+02, 3.163937598615976E+02, 2459856.859881772194
    od.getError([e,0,i,o,w,t,0,0,0,a])
    
    newdate = '2022-Jul-18 04:00:00'
    newjdtime = 2459778.6666667
    sunFile="/home/soonali/Desktop/SSP2022/1999GJ2SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 23 15.97',"\n\terror:",error(HMStoDeg(16,23,15.97),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+10 37 45.1',"\n\terror:",error(DMStoDeg(10,37,45.1),DMStoDeg(dec[0],dec[1],dec[2])))


testJPL()
    

Semi-major axis: 
	actual: 1.535501123505049 
	calculated: 1.5354211429515614 
	error: 0.00520875903399602
Eccentricity: 
	actual: 0.1980131722243141 
	calculated: 0.19778721613512643 
	error: 0.1141116455281651
Inclination: 
	actual: 11.27908127242095 
	calculated: 11.282346333234798 
	error: 0.028947932327010394
Longitude of Ascending Node: 
	actual: 196.197631075624 
	calculated: 196.2207864386297 
	error: 0.011802060442187455
Argument of perihelion: 
	actual: 142.5325538303718 
	calculated: 142.50990056939216 
	error: 0.015893394435770522
Time of Perihelion Passage T: 
	actual: 2459856.859881772 
	calculated: 2459161.9797162027 
	error: 0.0282488049163512
ra: 
	calculated: (16.0, 23.0, 14.353396357660131) 
	actual: 16 23 15.97 
	error: 0.002740193342595552
dec: 
	calculated: (10, 37, 34.53302251542514) 
	actual: +10 37 45.1 
	error: 0.02761518324682113


In [18]:
def obs4TestOD():
    file = "/home/soonali/Desktop/SSP2022/SoongInput.txt"
    data=Data()
    od=OD(file)
    od.MoG(selDate=['2022-06-28 04:33:44.089','2022-07-12 04:16:40.826','2022-07-14 04:41:39.025'])
    
    newdate = '2022-Jul-08 04:38:40.426'
    newjdtime = 2459768.6935234
    print("\n"+newdate)
    sunFile="/home/soonali/Desktop/SSP2022/Obs4SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 23 41.90',"\n\terror:",error(HMStoDeg(16,23,41.90),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+11 43 21.3',"\n\terror:",error(DMStoDeg(11,43,21.3),DMStoDeg(dec[0],dec[1],dec[2])))

    
    newdate = '2022-Jul-12 04:04:10.1640'
    newjdtime = 2459772.6695621
    print("\n"+newdate)
    sunFile="/home/soonali/Desktop/SSP2022/Obs4SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 23 54.01',"\n\terror:",error(HMStoDeg(16,23,54.01),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+11 23 27.2',"\n\terror:",error(DMStoDeg(11,23,27.2),DMStoDeg(dec[0],dec[1],dec[2])))

    newdate = '2022-Jul-14 04:31:37.7300'
    newjdtime = 2459774.6886311
    print("\n"+newdate)
    sunFile="/home/soonali/Desktop/SSP2022/Obs4SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 22 48.57',"\n\terror:",error(HMStoDeg(16,22,48.57),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+11 10 04.0',"\n\terror:",error(DMStoDeg(11,10,04.0),DMStoDeg(dec[0],dec[1],dec[2])))

    newdate = '2022-Jul-12 04:16:40.82'
    newjdtime = 2459772.6782503
    print("\n"+newdate)
    sunFile="/home/soonali/Desktop/SSP2022/Obs4SunPos.txt"
    ra,dec=od.ephemeris(newjdtime, newdate, sunFile)
    print("ra: \n\tcalculated:",ra,"\n\tactual:",'16 22 53.90',"\n\terror:",error(HMStoDeg(16,22,53.90),HMStoDeg(ra[0],ra[1],ra[2])))
    print("dec: \n\tcalculated:",dec, "\n\tactual:",'+11 23 23.8',"\n\terror:",error(DMStoDeg(11,23,23.8),DMStoDeg(dec[0],dec[1],dec[2])))

            
    
obs4TestOD()


2022-Jul-08 04:38:40.426
ra: 
	calculated: (16.0, 23.0, 39.59215829076584) 
	actual: 16 23 41.90 
	error: 0.003910144724647939
dec: 
	calculated: (11, 43, 10.335133317956435) 
	actual: +11 43 21.3 
	error: 0.025982296000457854

2022-Jul-12 04:04:10.1640
ra: 
	calculated: (16.0, 22.0, 51.88146392487624) 
	actual: 16 23 54.01 
	error: 0.10524193778319385
dec: 
	calculated: (11, 23, 15.771240299071444) 
	actual: +11 23 27.2 
	error: 0.027870129394169548

2022-Jul-14 04:31:37.7300
ra: 
	calculated: (16.0, 22.0, 46.54123422486009) 
	actual: 16 22 48.57 
	error: 0.003440418811474222
dec: 
	calculated: (11, 9, 52.21003634274043) 
	actual: +11 10 04.0 
	error: 0.029325349858865

2022-Jul-12 04:16:40.82
ra: 
	calculated: (16.0, 22.0, 51.77989736209838) 
	actual: 16 22 53.90 
	error: 0.003594984625233103
dec: 
	calculated: (11, 23, 12.585415971487969) 
	actual: +11 23 23.8 
	error: 0.02735010908381535


In [19]:
# Data class
class Data:
    '''Class that reads and interprets data from input file'''
    
    def __init__(self):
        """ Initializes the class
            Args:
                None
            Returns:
                None
        """ 
        self.info=np.array([]) # non formatted data from input file
        self.inputData=None # formatted data from input file (list)]
        
        self.infoByTime={}
        self.infoByDate={} # dictionary of values. date is key. format for key: 2018-Jul-14 00:00:00.0000
        self.sunFileName=""
        self.inputFileName=""
        
        # constants
        self.JDTIME=0
        self.DATE=1
        self.RA=2
        self.DEC=3
        self.R=4
        
    def getInput(self, file:str)->list:
        """ Formats and returns formatted input. Stores into dictionaries, and converts all RA and Dec 
            into decimals. 
            Args:
                file (str): the input file name
            Returns:
                list: the formatted input [jdtime (float), date (str), ra (float), dec (float), [RX,RY,RZ] (np.array)]
        """ 
        self.inputFileName=file
        self.info=np.loadtxt(file,dtype=str,delimiter=",")
        # store in dictionary for fast retrieval; also, formats all the dec and ra, converting to decimals
        # [jdtime, date(str), ra, dec, [RX,RY,RZ] (np.array)]
        self.inputData=[]

        for line in range(1,len(self.info)):
            data=self.info[line]
            jdtime = float(data[0])
            date = data[1].strip()
            
            strRA,strDec = data[2].split(':'), data[3].split(':')
            h,m,s=float(strRA[0]),float(strRA[1]),float(strRA[2])
            ra=HMStoDeg(h,m,s)
            d,m,s=float(strDec[0]),float(strDec[1]),float(strDec[2])
            dec=DMStoDeg(d,m,s)
            
            R=[float(data[4]),float(data[5]),float(data[6])]
            
            self.inputData.append([jdtime, date, ra, dec, R])
            self.infoByDate[date] = [jdtime, date, ra, dec, R]
            self.infoByTime[jdtime] = [jdtime, date, ra, dec, R]
            
        return self.inputData # nothing is formatted, just all information
        
    def getSunInput(self,date:str=None,time:float=None)->list:
        """ Returns the sun input for a given time
            Args:
                date (str): optional; the date for sun vector
                time (float): optional; the time in Julian days for sun vector
            Returns:
                list: the sun vector
        """ 
        if np.shape(self.info)==0: raise Exception("No input has been loaded up")
        if not(date==None):
            return self.infoByDate[date][self.R]
            
        elif not(time==None):
            return self.infoByTime[time][self.R]
        
        else: # nothing has been inputted, throw an exception
            raise Exception("No time has been given to find sun input")
        
    def getRADECInput(self,date:str=None,time:float=None):
        """ Returns the right ascension and declination for a given time
            Args:
                date (str): optional; the date for right ascension and declination
                time (float): optional; the time in Julian days for right ascension and declination
            Returns:
                floats: ra, dec
        """ 
        if np.shape(self.info)==0: raise Exception("No input has been loaded up")
        if not(date==None):
            d=self.infoByDate[date]
            return d[self.RA], d[self.DEC]
            
        elif not(time==None):
            d=self.infoByTime[time]
            return d[self.RA], d[self.DEC]
            
        else: # nothing has been inputted, throw an exception
            raise Exception("No time has been given to find ra")
            
    def getJDTime(self,date:str)->float:
        """ Returns Julian Days time given date
            Args:
                date (str): optional; the date for right ascension and declination
                time (float): optional; the time in Julian days for right ascension and declination
            Returns:
                floats: ra, dec
        """ 
        if np.shape(self.info)==0: raise Exception("No input has been loaded up")
        
        d=self.infoByDate[date]
        return d[self.JDTIME]

    
    def formatTestInputInfo(self, file:str):
        """ Returns re-formatted data from test input file (for testing purposes, specifically OD elements generation)
            Args:
                file (str): file name
            Returns:
                lists: time (in julian days), data [[x,y,z],[dx,dy,dz]], timestamps (strings)
        """
        info=np.loadtxt(file,dtype=str,delimiter=",")
        time=np.array([float(info[i,0]) for i in range(1,len(info))])
        timestamps=np.copy(info[1:,1])
        
        return time,np.array([([float(info[i][2]),float(info[i][3]),float(info[i][4])], 
                        [float(info[i][5]),float(info[i][6]),float(info[i][7])]) for i in range(1,len(info))]), timestamps
   
    def getTestInput(self, file:str, date:str):
        """ Returns pos, vel, and times for testing asteroid (for testing purposes, specifically OD elements generation)
            Args:
                file (str): file name
                date (str): date for testing
            Returns:
                lists: pos, vel, time
        """
        times,data,timestamps=self.formatTestInputInfo(file)           
        line = 0
        for i in range(len(timestamps)):
            if date in timestamps[i]: 
                line = i
                break
                
        time,info=times[line],data[line]
        pos,vel=info[0],info[1]
        return pos, vel, time
                               
    def getODActualData(self, file:str, date:str)->list:
        """ Returns the actual values for OD elements given file and date 
            Args:
                file (str): file name
                date (str): the date
            Returns:
                list: the actual OD elements
        """
        info=np.loadtxt(file,dtype=str,delimiter=",")
        timestamps=info[1:,1]
    
        flag=False
        for i in range(len(timestamps)):
            if date in timestamps[i]: 
                line = i
                flag=True
                break
        if not flag: raise Exception("Actual OD elements results not found in file")
        
        data = np.array([[float(info[i][j]) for j in range(2,14)] for i in range(1,len(info))])

        return data[line]
    
    def getSunPos(self, date:str, file:str)->list:
        """ Gets the vector from the Earth to the Sun given the date
            Args:
                date (str): the date to use
                file (str): file name
            Returns:
                list: sun vector R
        """
        info=np.loadtxt(file,dtype=str,delimiter=",")
        
        timestamps=info[:,1]
        flag=False
        for i in range(len(timestamps)):
            if date in timestamps[i]: 
                line = i
                flag=True
                break
        
        if not flag: return np.array([0,0,0]) # not found
        
        stuff=info[line,:]
        x,y,z=float(stuff[2]),float(stuff[3]),float(stuff[4])
        return np.array([x,y,z])
    
    def printResults(self, fileName:str, pos:list, vel:list, rho:list, a:float, e:float, i:float, o:float, T:float, w:float, date:str, M:float):
        """ Gets the vector from the Earth to the Sun given the date
            Args:
                fileName (str): exported file name
                pos (list): position from sun
                vel (list): the velocity of the asteroid
                rho (list): the position from Earth
                a (float): semi major axis
                e (float): eccentricity
                i (float): inclination
                o (float): longitude of ascending node (Degrees)
                T (float): Time of perihelion passage
                w (float): argument of perihelion (Degrees)
            Returns:
                None
        """
        with open(fileName, 'w') as file:
            file.write("1999 GJ2 Orbit Determination")
            file.write("\n----------------------------------------------\n")
            file.write("Position From Sun (r, AU):\n\t" + str(pos))
            file.write("\nVelocity (AU/day):\n\t"+str(vel))
            file.write("\nPosition From Earth (rho, AU):\n\t"+str(rho))
            file.write("\n----------------------------------------------\n")
            file.write("Orbital Elements:")
            file.write("\n\tSemi-Major Axis: " + str(a))
            file.write("\n\tEccentricity: " + str(e))
            file.write("\n\tInclination (deg): " + str(i))
            file.write("\n\tLongitude of Ascending Node (deg): " + str(o))
            file.write("\n\tTime of Perihelion Passage: " + str(T))
            file.write("\n\tArgument of Perihelion (deg): " + str(w))
            file.write("\n----------------------------------------------\n")
            file.write("Mean Anomaly for "+date+":")
            file.write("\n\t"+str(M))
            
        